# **한글 코딩 정보 벡터 DB화**

## 필요 패키지 다운로드

In [1]:
!pip install -q langchain langchain-openai langchain-ollama langchain-community langchain-chroma langchain-text-splitters tiktoken huggingface_hub sentence_transformers pypdf grandalf


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


## OpenAI API Key Setup

In [ ]:
import os

In [2]:
import os
from langchain.schema import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [5]:
# 데이터 파일이 위치한 폴더 경로
data_folder = 'data/codebook'

# 001.md부터 028.md 파일까지 읽어서 처리할 문서 리스트
docs = []
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

for i in range(1, 29):
    file_name = f"{i:03d}.md"
    file_path = os.path.join(data_folder, file_name)
    if not os.path.exists(file_path):
        print(f"파일이 존재하지 않습니다: {file_path}")
        continue

    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    # 텍스트 청크로 분할 후 Document 객체로 변환
    #text_chunks = text_splitter.split_text(content)
    file_docs = [Document(page_content=content)]
    docs.extend(file_docs)

print(f"총 Document 개수: {len(docs)}")

# ChromaDB에 청크들을 벡터 임베딩으로 저장(OpenAI 임베딩 모델 활용)

embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",          # 최신 텍스트 전용 임베딩 모델
    task_type="RETRIEVAL_DOCUMENT"         # 필요 시 TASK 타입 지정
)

vectorstore = Chroma.from_documents(docs, OpenAIEmbeddings(model='text-embedding-3-small'), persist_directory="./chroma_db20")
retriever = vectorstore.as_retriever()

총 Document 개수: 28


Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [9]:
import os
from PyPDF2 import PdfReader                       # 추가
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

# 데이터 파일이 위치한 폴더 경로
data_folder = 'data/codebook'

docs = []

for i in range(1, 3):
    file_name = f"{i:03d}.pdf"                     # md → pdf
    file_path = os.path.join(data_folder, file_name)
    if not os.path.exists(file_path):
        print(f"파일이 존재하지 않습니다: {file_path}")
        continue

    # PDF 텍스트 추출
    reader = PdfReader(file_path)
    content = ""
    for page in reader.pages:                      # 페이지별 텍스트 결합
        text = page.extract_text()
        if text:
            content += text + "\n"

    # 텍스트를 청크로 분할
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=100
    )
    text_chunks = text_splitter.split_text(content)

    # Document 객체 생성 + 메타데이터(source)
    file_docs = [Document(page_content=chunk, metadata={"source": file_name})
                 for chunk in text_chunks]

    docs.extend(file_docs)

print(f"총 Document 개수: {len(docs)}")


def batched(seq, size):
    for i in range(0, len(seq), size):
        yield seq[i:i + size]

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 빈 VectorStore 먼저 생성
vectorstore = Chroma(
    persist_directory="./chroma_db30",
    embedding_function=embeddings
)

# 100개(또는 토큰 기준 안전한 크기)씩 나눠서 추가
for batch in batched(docs, 100):
    vectorstore.add_documents(batch)


retriever = vectorstore.as_retriever()


/var/folders/00/b48zl4b12fv_c52sqg6957lw0000gn/T/ipykernel_31903/1213623711.py:51: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(
Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


총 Document 개수: 369


In [11]:
retriever.get_relevant_documents("4학년 사회 과목의 평가 방법은?")

[Document(metadata={'source': '001.pdf'}, page_content='들이 실제로 탐구하는 과정을 평가할 필요가 있다. \n(나) 사회문제 탐구 과목은 사회문제에 지속적으로 관심을 가지고 합리적인 해결 방안을 모색하\n여 참여할 수 있는 시민을 기르는 데 목적이 있으므로 가치 및 태도 함양 여부도 평가할 \n필요가 있다.\n(다) 비판적 사고력 , 문제 해결 능력 및 의사 결정 능력, 의사소통 및 협업 능력, 정보 수집 및 \n분석 능력 등을 평가하기 위해 다양한 사회문제의 실태 조사, 원인 파악, 문제 해결을 위한 \n정보 수집과 분석, 적절한 해결 방안의 탐구 및 토의 활동 전반을 보고서법이나 프로젝트 법, \n토의법 등을 활용하여 평가할 필요가 있다. \n사회과 교육과정\n284(라) 충분한 시간을 가지고 학습 과정을 중시하는 평가가 이루어지도록 한다. 평가 과정에서 학생\n들이 학습 목표가 무엇이고 , 현재 자신의 상태는 어느 정도이며 , 어떻게 개선해 나아가야 \n하는지에 대해 지속해서 생각하게 함으로써 학생의 성장을 돕고 수준별 학습 지도에 활용한다 . \n(마) 느린 학습자 , 최소 성취수준 미도달이 예상되는 학생 등 다양한 학습자를 고려하여 학생의 \n수준에 따른 평가를 실시하기 위하여 디지털 인프라 및 원격수업 평가 도구 등을 활용하여 \n온오프라인을 연계한 평가를 실시할 수 있다. \n(2) 평가 방법\n(가) 보고서법이나 프로젝트법 등을 활용할 때에는 내용의 타당성과 충실성을 중심으로 평가한다 . \n이때 사회문제의 원인 파악이 제대로 이루어졌는지 , 해결 방안이 문제의 원인과 연계되어 \n있는지 , 적절하고 실현 가능한지에 초점을 둔다.\n(나) 수업 중 보고서 작성 활동을 진행하면서 서론, 선행 연구 검토, 탐구 계획 설계, 연구 결과 \n분석, 결론 등을 작성하게 하고 구체적인 체크리스트를 통해 학생들의 탐구 활동 과정을 \n단계별로 평가할 수 있다. 또는 탐구 주제를 선정한 이후 탐구 계획서 작성, 자료

In [ ]:
#GPT 3.5 모델 선언
from langchain import hub
llm = ChatOpenAI(model="gpt-4o-mini")

#Langchain Hub에서 RAG 프롬프트 호출
prompt = hub.pull("rlm/rag-prompt")

#Retriever로 검색한 유사 문서의 내용을 하나의 string으로 결합
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
answer = rag_chain.invoke("한글 프로그래밍 언어에서 조건문, 반복문, 변수 문법은 어떻게 사용해야 하는지 500자 이상의 자세한 예시를 들어 설명해? 출처 Document의 메타데이터 중 'grammar' 값을 출처명으로 설명 결과와 출력해줘.")
print(answer)

In [ ]:
result = retriever.get_relevant_documents("반복문과 조건문과 변수")
for i in range(len(result)):
    print(f"{i+1}번째 유사 청크")
    print(result[i].page_content[:200])
    print("-"*100)

In [ ]:
vectorstore.similarity_search_with_score("반복문과 조건문과 변수")